# Setup

### Mount Drive

In [3]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


### Path to training data on drive

In [4]:
path_folder = "/content/gdrive/My Drive/Colab Notebooks/Project/zerowaste-f-final/splits_final_deblurred/"

### Imports

In [5]:
from __future__ import print_function
import tensorflow as tf
from tensorflow import keras
from keras.datasets import mnist
#import tensorflow.keras.backend as K
from matplotlib import pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Flatten, Activation, concatenate, BatchNormalization
import matplotlib.pyplot as plt
import numpy as np
from keras.activations import softmax
from keras.layers import concatenate
from keras.layers import UpSampling2D
from keras.layers import Dropout
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Input
from keras.models import Model
import sys
import cv2
sys.path.append("/content/gdrive/My Drive/Colab Notebooks/Project/")

import os
from PIL import Image

In [6]:
def load_data_for_split(split='data', amount_to_load=100, load_from_image=0):
    file_path = path_folder + split + "/"
    data = []

    if not os.path.exists(file_path):
        print(f"Error: The specified path '{file_path}' does not exist.")
        return None

    # List all files in the directory
    file_list = sorted(os.listdir(file_path))
    file_list = file_list[load_from_image:load_from_image + amount_to_load]

    for i, file_name in enumerate(file_list):
        if i >= amount_to_load:
            break

        image_path = os.path.join(file_path, file_name)
        try:
            # Open and load the image
            img = Image.open(image_path)
            data.append(np.array(img))
            if i%100==0 or i==amount_to_load-1:
              print(f"Loaded image number {load_from_image + i}: {image_path}")
        except Exception as e:
            print(f"Error loading {image_path}: {str(e)}")
    data = np.array(data)
    return data

def percentage_pixel_imbalance_for_images(labels):
    total_pixels = np.sum(labels[0] >= 0) *  len(labels)
    total_percentages  = {
        'background': 0,
        'rigid_plastic': 0,
        'cardboard': 0,
        'metal': 0,
        'soft_plastic': 0
    }

    for label in labels:
        total_percentages['background'] += np.sum(label == 0)
        total_percentages['rigid_plastic'] += np.sum(label == 1)
        total_percentages['cardboard'] += np.sum(label == 2)
        total_percentages['metal'] += np.sum(label == 3)
        total_percentages['soft_plastic'] += np.sum(label == 4)

    for class_name in total_percentages:
        total_percentages[class_name] /= total_pixels
        total_percentages[class_name] *= 100
        total_percentages[class_name] = np.round(total_percentages[class_name], 2)

    return total_percentages


In [7]:
loaded_train_labels = load_data_for_split('train/sem_seg', amount_to_load=1500, load_from_image=0)
train_percentage1 = percentage_pixel_imbalance_for_images(loaded_train_labels)
loaded_train_labels = load_data_for_split('train/sem_seg', amount_to_load=1502, load_from_image=1500)
train_percentage2 = percentage_pixel_imbalance_for_images(loaded_train_labels)
loaded_test_labels = load_data_for_split('test/sem_seg', 3500)
loaded_val_labels = load_data_for_split('val/sem_seg', 3500)

Loaded image number 0: /content/gdrive/My Drive/Colab Notebooks/Project/zerowaste-f-final/splits_final_deblurred/train/sem_seg/01_frame_001160.PNG
Loaded image number 100: /content/gdrive/My Drive/Colab Notebooks/Project/zerowaste-f-final/splits_final_deblurred/train/sem_seg/02_frame_001620.PNG
Loaded image number 200: /content/gdrive/My Drive/Colab Notebooks/Project/zerowaste-f-final/splits_final_deblurred/train/sem_seg/03_frame_000611.PNG
Loaded image number 300: /content/gdrive/My Drive/Colab Notebooks/Project/zerowaste-f-final/splits_final_deblurred/train/sem_seg/03_frame_001610.PNG
Loaded image number 400: /content/gdrive/My Drive/Colab Notebooks/Project/zerowaste-f-final/splits_final_deblurred/train/sem_seg/04_frame_003600.PNG
Loaded image number 500: /content/gdrive/My Drive/Colab Notebooks/Project/zerowaste-f-final/splits_final_deblurred/train/sem_seg/04_frame_029900.PNG
Loaded image number 600: /content/gdrive/My Drive/Colab Notebooks/Project/zerowaste-f-final/splits_final_deb

In [8]:
train_percentage_final  = {
        'background': 0,
        'rigid_plastic': 0,
        'cardboard': 0,
        'metal': 0,
        'soft_plastic': 0
    }
average_percentage_final  = {
        'background': 0,
        'rigid_plastic': 0,
        'cardboard': 0,
        'metal': 0,
        'soft_plastic': 0
    }


print("Train pixel imbalance")
for class_name in train_percentage1:
    train_percentage_final[class_name] = np.round((train_percentage1[class_name] + train_percentage2[class_name]) / 2, 2)
print(train_percentage_final)

print("\nVal pixel imbalance")
val_percentage_final = percentage_pixel_imbalance_for_images(loaded_val_labels)
print(val_percentage_final)

print("\nTest pixel imbalance")
test_percentage_final = percentage_pixel_imbalance_for_images(loaded_test_labels)
print(test_percentage_final)

print("\nAverage pixel imbalance")
average_percentage_final = percentage_pixel_imbalance_for_images(loaded_val_labels)
for class_name in train_percentage1:
    average_percentage_final[class_name] = np.round((train_percentage_final[class_name] + val_percentage_final[class_name] + test_percentage_final[class_name]) / 3, 2)
print(average_percentage_final)

Train pixel imbalance
{'background': 83.74, 'rigid_plastic': 0.63, 'cardboard': 10.36, 'metal': 0.09, 'soft_plastic': 5.17}

Val pixel imbalance
{'background': 81.28, 'rigid_plastic': 1.09, 'cardboard': 11.79, 'metal': 0.18, 'soft_plastic': 5.66}

Test pixel imbalance
{'background': 82.03, 'rigid_plastic': 0.79, 'cardboard': 11.38, 'metal': 0.07, 'soft_plastic': 5.74}

Average pixel imbalance
{'background': 82.35, 'rigid_plastic': 0.84, 'cardboard': 11.18, 'metal': 0.11, 'soft_plastic': 5.52}
